# Machine Learning Classification Models

**DOST-ITDI AI Training Workshop**  
**Day 1 - Session 3 (continued): Classification with Scikit-learn**

---

## Learning Objectives
1. Understand classification problems in chemistry
2. Implement classification algorithms
3. Evaluate models using classification metrics
4. Interpret confusion matrices and ROC curves
5. Handle imbalanced datasets

## What is Classification?

Classification predicts **categorical labels** (classes).

**Chemistry Applications**:
- Drug activity (active/inactive)
- Toxicity prediction (toxic/non-toxic)
- Compound type classification
- Quality control (pass/fail)
- Reaction outcome (success/failure)

## 1. Setup and Data Loading

In [ ]:
# Install libraries
!pip install rdkit scikit-learn imbalanced-learn -q

print("✓ Installation complete!")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Descriptors
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

# Set random seed
np.random.seed(42)

# Plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("✓ Libraries imported successfully!")

In [ ]:
# Load BACE dataset (Blood-Brain Barrier Penetration)
# Binary classification: active (1) vs inactive (0)
url = "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/bace.csv"
df = pd.read_csv(url)

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
# Check target distribution - DETAILED ANALYSIS
print("="*60)
print("CLASS BALANCE ANALYSIS")
print("="*60)

# Absolute counts
print("\n1. Class Counts:")
class_counts = df['Class'].value_counts().sort_index()
print(class_counts)

# Percentages
print("\n2. Class Percentages:")
class_percentages = df['Class'].value_counts(normalize=True).sort_index() * 100
for cls, pct in class_percentages.items():
    print(f"   Class {cls}: {pct:.2f}%")

# Ratio
ratio = class_counts[0] / class_counts[1]
print(f"\n3. Majority to Minority Ratio: {ratio:.2f}:1")

# Interpretation guidance
print(f"\n4. Interpretation:")
print(f"   This dataset has a ratio of {ratio:.2f}:1")
print(f"   Generally considered balanced (close to 1:1)")
print(f"   Imbalance becomes a concern when ratio > 3:1 or 5:1")
print(f"   Severe imbalance: 10:1, 100:1, or higher")
print(f"   (Thresholds depend on problem domain and cost of errors)")

print("\n" + "="*60)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
class_counts.plot(kind='bar', ax=axes[0], color=['coral', 'steelblue'], 
                  edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Class (0=Inactive, 1=Active)', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Class Distribution (Counts)', fontsize=14, fontweight='bold')
axes[0].set_xticklabels(['Inactive (0)', 'Active (1)'], rotation=0)
axes[0].grid(True, alpha=0.3, axis='y')

# Add count labels on bars
for i, (idx, val) in enumerate(class_counts.items()):
    axes[0].text(i, val + 20, str(val), ha='center', fontsize=12, fontweight='bold')

# Pie chart
axes[1].pie(class_counts.values, labels=['Inactive (0)', 'Active (1)'],
            autopct='%1.1f%%', startangle=90, colors=['coral', 'steelblue'],
            explode=[0.05, 0.05], shadow=True)
axes[1].set_title('Class Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 2. Feature Engineering

In [ ]:
# Compute molecular descriptors
df['mol'] = df['mol'].apply(Chem.MolFromSmiles)
df = df[df['mol'].notna()].copy()

# Calculate descriptors
df['MolWeight'] = df['mol'].apply(Descriptors.MolWt)
df['LogP'] = df['mol'].apply(Descriptors.MolLogP)
df['NumHDonors'] = df['mol'].apply(Descriptors.NumHDonors)
df['NumHAcceptors'] = df['mol'].apply(Descriptors.NumHAcceptors)
df['NumRotatableBonds'] = df['mol'].apply(Descriptors.NumRotatableBonds)
df['NumAromaticRings'] = df['mol'].apply(Descriptors.NumAromaticRings)
df['TPSA'] = df['mol'].apply(Descriptors.TPSA)
df['NumAtoms'] = df['mol'].apply(lambda x: x.GetNumAtoms())

print(f"Dataset after feature engineering: {df.shape[0]} molecules")
print("\nNew features:")
print(df[['MolWeight', 'LogP', 'NumHDonors', 'NumHAcceptors']].head())

In [ ]:
# Feature distribution by class
features_to_plot = ['MolWeight', 'LogP', 'NumHDonors', 'TPSA']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, feature in enumerate(features_to_plot):
    df.boxplot(column=feature, by='Class', ax=axes[idx], patch_artist=True)
    axes[idx].set_xlabel('Class', fontsize=11)
    axes[idx].set_ylabel(feature, fontsize=11)
    axes[idx].set_title(f'{feature} by Class', fontsize=12, fontweight='bold')
    axes[idx].get_figure().suptitle('')

plt.tight_layout()
plt.show()

## 3. Data Preparation

In [ ]:
# Select features and target
feature_columns = ['MolWeight', 'LogP', 'NumHDonors', 'NumHAcceptors', 
                   'NumRotatableBonds', 'NumAromaticRings', 'TPSA', 'NumAtoms']

X = df[feature_columns]
y = df['Class']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nClass distribution:")
print(y.value_counts())

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # stratify maintains class balance
)

print("Dataset Split:")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTrain class distribution:")
print(y_train.value_counts())
print(f"\nTest class distribution:")
print(y_test.value_counts())

In [ ]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Feature scaling complete!")

## 4. Classification Models

### 4.1 Logistic Regression

In [ ]:
# Train Logistic Regression
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Predictions
y_train_pred_lr = log_reg.predict(X_train_scaled)
y_test_pred_lr = log_reg.predict(X_test_scaled)
y_test_proba_lr = log_reg.predict_proba(X_test_scaled)[:, 1]  # Probability of class 1

# Evaluate
print("Logistic Regression Results:")
print(f"Training Accuracy: {accuracy_score(y_train, y_train_pred_lr):.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred_lr):.4f}")
print(f"Test Precision: {precision_score(y_test, y_test_pred_lr):.4f}")
print(f"Test Recall: {recall_score(y_test, y_test_pred_lr):.4f}")
print(f"Test F1-Score: {f1_score(y_test, y_test_pred_lr):.4f}")
print(f"Test ROC-AUC: {roc_auc_score(y_test, y_test_proba_lr):.4f}")

### 4.2 Random Forest Classifier

In [ ]:
# Train Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_clf.fit(X_train_scaled, y_train)

# Predictions
y_train_pred_rf = rf_clf.predict(X_train_scaled)
y_test_pred_rf = rf_clf.predict(X_test_scaled)
y_test_proba_rf = rf_clf.predict_proba(X_test_scaled)[:, 1]

# Evaluate
print("Random Forest Results:")
print(f"Training Accuracy: {accuracy_score(y_train, y_train_pred_rf):.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred_rf):.4f}")
print(f"Test Precision: {precision_score(y_test, y_test_pred_rf):.4f}")
print(f"Test Recall: {recall_score(y_test, y_test_pred_rf):.4f}")
print(f"Test F1-Score: {f1_score(y_test, y_test_pred_rf):.4f}")
print(f"Test ROC-AUC: {roc_auc_score(y_test, y_test_proba_rf):.4f}")

In [ ]:
# Feature importance
feature_imp = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': rf_clf.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_imp['Feature'], feature_imp['Importance'], 
         color='steelblue', alpha=0.7, edgecolor='black')
plt.xlabel('Importance Score', fontsize=12)
plt.title('Random Forest: Feature Importance', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### 4.3 Gradient Boosting Classifier

In [ ]:
# Train Gradient Boosting
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_clf.fit(X_train_scaled, y_train)

# Predictions
y_train_pred_gb = gb_clf.predict(X_train_scaled)
y_test_pred_gb = gb_clf.predict(X_test_scaled)
y_test_proba_gb = gb_clf.predict_proba(X_test_scaled)[:, 1]

# Evaluate
print("Gradient Boosting Results:")
print(f"Training Accuracy: {accuracy_score(y_train, y_train_pred_gb):.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred_gb):.4f}")
print(f"Test Precision: {precision_score(y_test, y_test_pred_gb):.4f}")
print(f"Test Recall: {recall_score(y_test, y_test_pred_gb):.4f}")
print(f"Test F1-Score: {f1_score(y_test, y_test_pred_gb):.4f}")
print(f"Test ROC-AUC: {roc_auc_score(y_test, y_test_proba_gb):.4f}")

## 5. Model Comparison

In [ ]:
# Compile results
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'Gradient Boosting'],
    'Accuracy': [
        accuracy_score(y_test, y_test_pred_lr),
        accuracy_score(y_test, y_test_pred_rf),
        accuracy_score(y_test, y_test_pred_gb)
    ],
    'Precision': [
        precision_score(y_test, y_test_pred_lr),
        precision_score(y_test, y_test_pred_rf),
        precision_score(y_test, y_test_pred_gb)
    ],
    'Recall': [
        recall_score(y_test, y_test_pred_lr),
        recall_score(y_test, y_test_pred_rf),
        recall_score(y_test, y_test_pred_gb)
    ],
    'F1-Score': [
        f1_score(y_test, y_test_pred_lr),
        f1_score(y_test, y_test_pred_rf),
        f1_score(y_test, y_test_pred_gb)
    ],
    'ROC-AUC': [
        roc_auc_score(y_test, y_test_proba_lr),
        roc_auc_score(y_test, y_test_proba_rf),
        roc_auc_score(y_test, y_test_proba_gb)
    ]
})

print("\nModel Performance Comparison:")
print(results.to_string(index=False))

# Best model
best_model_idx = results['ROC-AUC'].idxmax()
print(f"\n🏆 Best Model: {results.loc[best_model_idx, 'Model']}")
print(f"   ROC-AUC: {results.loc[best_model_idx, 'ROC-AUC']:.4f}")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Metrics comparison
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(results))
width = 0.2

for i, metric in enumerate(metrics_to_plot):
    axes[0].bar(x + i*width, results[metric], width, label=metric, alpha=0.8)

axes[0].set_xlabel('Model', fontsize=12)
axes[0].set_ylabel('Score', fontsize=12)
axes[0].set_title('Classification Metrics Comparison', fontsize=13, fontweight='bold')
axes[0].set_xticks(x + width * 1.5)
axes[0].set_xticklabels(results['Model'], rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# ROC-AUC comparison
axes[1].bar(results['Model'], results['ROC-AUC'], 
            color='coral', alpha=0.8, edgecolor='black')
axes[1].set_xlabel('Model', fontsize=12)
axes[1].set_ylabel('ROC-AUC Score', fontsize=12)
axes[1].set_title('ROC-AUC Comparison', fontsize=13, fontweight='bold')
axes[1].set_xticklabels(results['Model'], rotation=45, ha='right')
axes[1].set_ylim([0.5, 1.0])
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6. Confusion Matrix

In [ ]:
# Confusion matrices for all models
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

predictions = [
    ('Logistic Regression', y_test_pred_lr),
    ('Random Forest', y_test_pred_rf),
    ('Gradient Boosting', y_test_pred_gb)
]

for idx, (name, y_pred) in enumerate(predictions):
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[idx],
                xticklabels=['Inactive', 'Active'],
                yticklabels=['Inactive', 'Active'])
    axes[idx].set_xlabel('Predicted', fontsize=11)
    axes[idx].set_ylabel('Actual', fontsize=11)
    axes[idx].set_title(f'{name}', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Detailed confusion matrix explanation for Random Forest
cm_rf = confusion_matrix(y_test, y_test_pred_rf)

tn, fp, fn, tp = cm_rf.ravel()

print("Confusion Matrix Breakdown (Random Forest):")
print(f"True Negatives (TN): {tn} - Correctly predicted as Inactive")
print(f"False Positives (FP): {fp} - Incorrectly predicted as Active")
print(f"False Negatives (FN): {fn} - Incorrectly predicted as Inactive")
print(f"True Positives (TP): {tp} - Correctly predicted as Active")
print(f"\nTotal predictions: {tn + fp + fn + tp}")
print(f"Correct predictions: {tn + tp} ({(tn + tp)/(tn + fp + fn + tp)*100:.2f}%)")
print(f"Incorrect predictions: {fp + fn} ({(fp + fn)/(tn + fp + fn + tp)*100:.2f}%)")

## 7. ROC Curve Analysis

In [ ]:
# Plot ROC curves for all models
plt.figure(figsize=(10, 8))

models_proba = [
    ('Logistic Regression', y_test_proba_lr),
    ('Random Forest', y_test_proba_rf),
    ('Gradient Boosting', y_test_proba_gb)
]

for name, y_proba in models_proba:
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    plt.plot(fpr, tpr, linewidth=2, label=f'{name} (AUC = {auc:.3f})')

# Random classifier line
plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier (AUC = 0.5)')

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.show()

## 8. Classification Report

In [ ]:
# Detailed classification report for Random Forest
print("Classification Report (Random Forest):")
print("=" * 60)
print(classification_report(y_test, y_test_pred_rf, 
                           target_names=['Inactive (0)', 'Active (1)']))

## 9. Understanding Classification Metrics

### Key Metrics Explained:

1. **Accuracy**: (TP + TN) / Total
   - Overall correctness
   - Can be misleading with imbalanced data

2. **Precision**: TP / (TP + FP)
   - Of all positive predictions, how many are correct?
   - Important when false positives are costly

3. **Recall (Sensitivity)**: TP / (TP + FN)
   - Of all actual positives, how many did we find?
   - Important when false negatives are costly

4. **F1-Score**: 2 × (Precision × Recall) / (Precision + Recall)
   - Harmonic mean of precision and recall
   - Balanced measure

5. **ROC-AUC**: Area Under the ROC Curve
   - Overall model discrimination ability
   - Threshold-independent

### When to use which metric?

- **Drug Discovery**: High recall (don't miss active compounds)
- **Toxicity Screening**: High precision (avoid false alarms)
- **Quality Control**: Balance with F1-score
- **Model Comparison**: ROC-AUC

## 10. Prediction Threshold Optimization

In [ ]:
# Try different thresholds
thresholds = np.arange(0.1, 1.0, 0.05)
precision_scores = []
recall_scores = []
f1_scores = []

for threshold in thresholds:
    y_pred_threshold = (y_test_proba_rf >= threshold).astype(int)
    precision_scores.append(precision_score(y_test, y_pred_threshold))
    recall_scores.append(recall_score(y_test, y_pred_threshold))
    f1_scores.append(f1_score(y_test, y_pred_threshold))

# Plot
plt.figure(figsize=(10, 6))
plt.plot(thresholds, precision_scores, 'o-', label='Precision', linewidth=2)
plt.plot(thresholds, recall_scores, 's-', label='Recall', linewidth=2)
plt.plot(thresholds, f1_scores, '^-', label='F1-Score', linewidth=2)
plt.axvline(x=0.5, color='gray', linestyle='--', linewidth=1, label='Default (0.5)')
plt.xlabel('Classification Threshold', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.title('Metrics vs Classification Threshold', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

# Find optimal threshold for F1
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
print(f"\nOptimal threshold for F1-Score: {optimal_threshold:.2f}")
print(f"F1-Score at optimal threshold: {f1_scores[optimal_idx]:.4f}")

## 11. Making Predictions

In [ ]:
# Sample predictions
sample_indices = [0, 10, 20, 30, 40]
samples = X_test.iloc[sample_indices]
samples_scaled = scaler.transform(samples)

# Predict with probabilities
predictions = rf_clf.predict(samples_scaled)
probabilities = rf_clf.predict_proba(samples_scaled)
actual = y_test.iloc[sample_indices].values

# Display results
results_df = pd.DataFrame({
    'Actual': ['Active' if x == 1 else 'Inactive' for x in actual],
    'Predicted': ['Active' if x == 1 else 'Inactive' for x in predictions],
    'Prob_Inactive': probabilities[:, 0],
    'Prob_Active': probabilities[:, 1],
    'Correct': actual == predictions
})

print("\nSample Predictions:")
print(results_df.to_string())

In [ ]:
# Create Imbalanced Baseline Dataset
from imblearn.under_sampling import RandomUnderSampler

print("="*60)
print("CREATING IMBALANCED BASELINE DATASET")
print("="*60)

print("\nOriginal Training Data:")
print(f"Training set size: {X_train_scaled.shape[0]}")
class_counts_before = y_train.value_counts().sort_index()
print(f"Class distribution:\n{class_counts_before}")
ratio_before = class_counts_before[0] / class_counts_before[1]
print(f"Ratio: {ratio_before:.2f}:1 (relatively balanced)")

# Apply under-sampling - Remove 50% of majority class (Class 0)
# This creates an imbalanced dataset for demonstrating handling techniques
majority_class = class_counts_before.idxmax()
majority_count = class_counts_before[majority_class]
target_majority_count = int(majority_count * 0.5)

print(f"\nCreating Imbalance: Remove 50% of Class {majority_class}")
print(f"  Original: {majority_count} samples")
print(f"  Target: {target_majority_count} samples")
print(f"  Removing: {majority_count - target_majority_count} samples")

rus = RandomUnderSampler(sampling_strategy={majority_class: target_majority_count}, random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_scaled, y_train)

print(f"\nImbalanced Baseline Dataset Created:")
print(f"Training set size: {X_train_rus.shape[0]}")
class_counts_after = pd.Series(y_train_rus).value_counts().sort_index()
print(f"Class distribution:\n{class_counts_after}")
ratio_after = class_counts_after[1] / class_counts_after[0]
print(f"Ratio: {ratio_after:.2f}:1 (Class 1 is now majority)")
print(f"\nThis imbalanced dataset will be used to demonstrate")
print(f"different techniques for handling class imbalance.")
print("="*60)

In [ ]:
# Compare all techniques
print("\n" + "="*60)
print("COMPARISON OF ALL TECHNIQUES")
print("="*60)

print("\nAll models trained on same imbalanced baseline (328:553)")
print("Evaluated on same test set\n")

techniques = {
    'No Handling': y_pred_default,
    'Class Weights': y_pred_balanced,
    'SMOTE': y_pred_smote
}

comparison_results = []

for name, y_pred in techniques.items():
    comparison_results.append({
        'Technique': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    })

comparison_df = pd.DataFrame(comparison_results)
print("\nComparison of Imbalance Handling Techniques:")
print("="*80)
print(comparison_df.to_string(index=False))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Precision vs Recall scatter
axes[0].scatter(comparison_df['Recall'], comparison_df['Precision'],
               s=200, alpha=0.6, c=range(len(comparison_df)),
               cmap='viridis', edgecolors='black', linewidth=2)
for idx, row in comparison_df.iterrows():
    axes[0].annotate(row['Technique'],
                    (row['Recall'], row['Precision']),
                    xytext=(5, 5), textcoords='offset points', fontsize=9)
axes[0].set_xlabel('Recall', fontsize=12)
axes[0].set_ylabel('Precision', fontsize=12)
axes[0].set_title('Precision vs Recall Trade-off', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# F1-Score comparison
axes[1].barh(comparison_df['Technique'], comparison_df['F1-Score'],
            color='steelblue', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('F1-Score', fontsize=12)
axes[1].set_title('F1-Score by Technique', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("- All techniques trained on same imbalanced baseline (328:553 = 1.69:1 ratio)")
print("- SMOTE and Class Weights help handle the imbalance")
print("- No Handling shows baseline performance without any technique")
print("- Compare Recall for minority class (Class 0) - higher is better")
print("- For more severe imbalance (10:1, 100:1), differences would be larger")
print("="*60)

### 12.5 Summary: Which Technique to Use?

| Technique | Pros | Cons | Best For |
|-----------|------|------|----------|
| **Class Weights** | Simple, no data change, fast | Limited for severe imbalance | First approach, moderate imbalance |
| **SMOTE** | Generates new data, maintains info | Can create noise | Good minority samples, SVM/NN |
| **Under-sampling** | Fast, reduces size | Loses information | Large datasets, time constraints |

**General Recommendations:**

1. **Start Simple:** Try class weights first
2. **Moderate Imbalance (3:1 to 10:1):** Class weights or SMOTE
3. **Severe Imbalance (>10:1):** SMOTE or combination methods
4. **Huge Dataset:** Under-sampling acceptable
5. **Always:** Use stratified cross-validation
6. **Metrics:** Focus on F1, precision, recall - not accuracy!

**For Chemistry/Drug Discovery:**
- **Toxicity screening:** Prioritize recall (don't miss toxic compounds)
- **Hit identification:** Balance precision/recall with F1-score
- **Rare effects:** Definitely need resampling techniques

### 12.4 Comparing All Techniques

In [ ]:
# Technique 1: SMOTE (Synthetic Minority Over-sampling Technique)
from imblearn.over_sampling import SMOTE

print("\n" + "="*60)
print("TECHNIQUE 1: SMOTE")
print("="*60)

print("\nStarting from Imbalanced Baseline:")
print(f"Training set size: {X_train_rus.shape[0]}")
baseline_counts = pd.Series(y_train_rus).value_counts().sort_index()
print(f"Class distribution:\n{baseline_counts}")
print(f"Ratio: {baseline_counts[1]/baseline_counts[0]:.2f}:1 (imbalanced)")

# Apply SMOTE to imbalanced baseline
print("\nApplying SMOTE...")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_rus, y_train_rus)

print(f"\nAfter SMOTE:")
print(f"Training set size: {X_train_smote.shape[0]}")
print(f"Class distribution:\n{pd.Series(y_train_smote).value_counts()}")
print(f"Ratio: 1:1 (perfectly balanced)")

# Train model on SMOTE data
rf_smote = RandomForestClassifier(n_estimators=100, random_state=42)
rf_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = rf_smote.predict(X_test_scaled)

print("\nModel Performance with SMOTE:")
print(classification_report(y_test, y_pred_smote,
                           target_names=['Inactive', 'Active']))

# Visualize SMOTE effect (2D projection)
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_baseline_pca = pca.fit_transform(X_train_rus)
X_smote_pca = pca.transform(X_train_smote)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Imbalanced Baseline
for class_val in [0, 1]:
    mask = y_train_rus == class_val
    axes[0].scatter(X_baseline_pca[mask, 0], X_baseline_pca[mask, 1],
                   label=f'Class {class_val}', alpha=0.6, s=30)
axes[0].set_title('Imbalanced Baseline (328:553)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('First Principal Component')
axes[0].set_ylabel('Second Principal Component')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# After SMOTE
for class_val in [0, 1]:
    mask = y_train_smote == class_val
    axes[1].scatter(X_smote_pca[mask, 0], X_smote_pca[mask, 1],
                   label=f'Class {class_val}', alpha=0.6, s=30)
axes[1].set_title('After SMOTE (Balanced)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('First Principal Component')
axes[1].set_ylabel('Second Principal Component')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("="*60)

### 12.3 Technique 3: Random Under-sampling

Remove samples from majority class to balance with minority class.

**How it works:**
- Randomly remove majority class samples
- Until both classes have similar size

**When to use:**
- Very large datasets where losing data is acceptable
- When training time is critical
- When majority class has redundant information

**Caution:**
- Loses potentially useful information
- May hurt model performance
- Only use if you have lots of majority samples

### Technique 1: SMOTE (Synthetic Minority Over-sampling Technique)

Generate synthetic samples of minority class by interpolating between existing samples.

**How it works:**
1. Select a minority sample
2. Find its k nearest neighbors
3. Create new sample between original and neighbor
4. Repeat until classes balanced

**When to use:**
- Moderate to severe imbalance
- When you need more training data
- Works well with SVM, Neural Networks

**In our demo:**
- Starts with imbalanced baseline (328:553)
- SMOTE creates synthetic samples for Class 0 (minority)
- Results in perfectly balanced dataset (553:553)

### 12.2 Technique 2: SMOTE (Synthetic Minority Over-sampling Technique)

Generate synthetic samples of minority class by interpolating between existing samples.

**How it works:**
1. Select a minority sample
2. Find its k nearest neighbors
3. Create new sample between original and neighbor
4. Repeat until classes balanced

**When to use:**
- Moderate to severe imbalance
- When you need more training data
- Works well with SVM, Neural Networks

**Caution:**
- May create synthetic samples in noisy regions
- Can lead to overfitting if overused

### Technique 2 & 3: No Handling (Baseline) and Class Weights

**Technique 2: No Handling (Baseline)**
- Train model directly on imbalanced data
- No special techniques applied
- Shows baseline performance for comparison

**Technique 3: Class Weights**
Automatically adjust model to penalize misclassification of minority class more heavily.

**How it works:**
- Assigns higher weight to minority class
- Model pays more attention to minority samples during training
- No data modification needed
- Formula: weight = n_samples / (n_classes * class_count)

**When to use:**
- First approach to try
- Moderate imbalance (up to 5:1)
- When you want to keep original data

**In our demo:**
- Both techniques use imbalanced baseline (328:553)
- Class weights: Class 0 gets ~1.34x weight, Class 1 gets ~0.80x weight
- Minority class errors penalized more during training

### 12.1 Technique 1: Class Weights

Automatically adjust model to penalize misclassification of minority class more heavily.

**How it works:**
- Assigns higher weight to minority class
- Model pays more attention to minority samples during training
- No data modification needed

**When to use:**
- First approach to try
- Moderate imbalance (up to 5:1)
- When you want to keep original data

## 12. Handling Imbalanced Datasets (Reference Guide)

### Note on This Dataset

Our BACE dataset is relatively balanced (54% vs 46%). However, in real-world chemistry applications, you'll often encounter imbalanced datasets:

**Common Imbalanced Scenarios:**
- Toxicity screening: 95% non-toxic, 5% toxic
- Rare adverse effects: 99% safe, 1% adverse
- Hit identification: 98% inactive, 2% active
- Quality control: 97% pass, 3% fail

This section shows techniques for handling such cases.

### What is Class Imbalance?

Class imbalance occurs when one class has significantly more samples than another.

**Problems it causes:**
- Model biased toward majority class
- Can achieve high accuracy by always predicting majority
- Poor detection of minority class (often the important one!)
- Misleading accuracy metric

## 12. Handling Imbalanced Datasets

### What is Class Imbalance?

Class imbalance occurs when one class has significantly more samples than another.

**Example:**
- Toxicity: 95% non-toxic, 5% toxic
- Rare diseases: 99% negative, 1% positive
- Quality control: 98% pass, 2% fail

**Problems:**
- Model biased toward majority class
- High accuracy but poor minority class detection
- Misleading performance metrics

## 12. Summary and Best Practices

### Key Takeaways:

1. **Data Preparation**
   - Check class balance
   - Use stratified split
   - Scale features appropriately

2. **Model Selection**
   - Start simple (Logistic Regression)
   - Try ensemble methods (Random Forest, Gradient Boosting)
   - Compare multiple models

3. **Evaluation**
   - Don't rely on accuracy alone
   - Use confusion matrix
   - Consider precision/recall trade-off
   - Use ROC-AUC for overall performance

4. **Chemistry-Specific Considerations**
   - Feature importance reveals key molecular properties
   - Threshold optimization depends on use case
   - Consider cost of false positives vs false negatives

### Next Steps:
- Deep Learning with PyTorch
- Neural networks for molecular data
- Transfer learning with transformers

## 13. Exercise

In [ ]:
# TODO: Try the following:
# 1. Train a Support Vector Machine (SVC) classifier
# 2. Compare its performance with the models above
# 3. Try adjusting the classification threshold
# 4. Create a confusion matrix for your model

# Your code here:


---

## Resources

- [Scikit-learn Classification Guide](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)
- [Understanding ROC Curves](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)
- [Precision vs Recall](https://en.wikipedia.org/wiki/Precision_and_recall)

**Next Notebook: Deep Learning with PyTorch**